<a href="https://colab.research.google.com/github/Alih111/naivee-and-logistic-nlp/blob/main/NLP_naivebayes_logisticRegression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets
from datasets import load_dataset
sst_dataset=load_dataset("sst")

getting training data sentences and labels


In [ ]:
train_data = sst_dataset["train"]
sentences=train_data["sentence"]
test_data = sst_dataset["test"]
test_sen=test_data['sentence']
test_score=test_data['label']
sentences=[s.lower() for s in sentences]
test_sen=[s.lower() for s in test_sen]
len(sentences)

8544

classify sentences according to score

•From 0 to 0.2 (0.2 included) will be class 0 “very negative”.
•From 0.2 to 0.4 (0.4 included) will be class 1 “negative”.
•From 0.4 to 0.6 (0.6included)will be class 2 “neutral”.•From 0.6to 0.8 (0.8 included)will be class 3 “positive”.•From 0.8to 1.0(1.0included)will be class 4 “very positive”.

In [ ]:
import numpy as np
scores= train_data["label"]
classes = [[] for _ in range(5)]
#put each sentence in the class that it belongs to
i=0
for score in scores:
    sentences[i]=sentences[i][:-1]
    if score <= 0.2:
      scores[i]=0
      classes[0].append(sentences[i])
    elif score > 0.2 and score <= 0.4:
      scores[i]=1
      classes[1].append(sentences[i])
    elif score > 0.4 and score <=0.6:
      scores[i]=2
      classes[2].append(sentences[i])
    elif score > 0.6 and score <=0.8:
      scores[i]=3
      classes[3].append(sentences[i])
    elif score > 0.8 and score <=1:
      scores[i]=4
      classes[4].append(sentences[i])
    i+=1
i=0
for score in test_score:
    if score <= 0.2:
      test_score[i]=0
    elif score > 0.2 and score <= 0.4:
      test_score[i]=1
    elif score > 0.4 and score <=0.6:
      test_score[i]=2
    elif score > 0.6 and score <=0.8:
      test_score[i]=3
    elif score > 0.8 and score <=1:
      test_score[i]=4
    i+=1
print(sentences[0])

the rock is destined to be the 21st century 's new `` conan '' and that he 's going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal 


count number of occurences of all words in the document


In [ ]:
from collections import Counter

sentences_np=np.array(sentences)
all_words = ' '.join(sentences_np).split()
word_counts = Counter(all_words)
word_counts_dict = dict(word_counts)
del word_counts
del sentences_np
del all_words
print(len(word_counts_dict))

16677


naivee bayes

In [ ]:
def countWord(goal,words):#count words in a single class
    return  np.sum(words == goal)

training loop


In [ ]:
from math import log
words_probablites={0: {},#will store the likelihood of each word in each class
                  1: {},
                   2: {},
                   3: {},4: {}}
Ndoc = sum(len(row) for row in classes)# counts the total number of words
print(Ndoc)
stopWords=['the' ,'is','are','you','he','she','it','they','a','am','an','that','to','and','or','be','can','of','was','with','on','his','her','from','at','as','in','can']
priors=[]
for k in range(len(classes)):
  Nc=len(classes[k])#count total number of words in each class
  priors.append(log(Nc/Ndoc))
  myclass = ' '.join(classes[k]).split()#get vocubulary for each class
  myclass = [word for word in myclass if word not in stopWords ]
  word_counts = Counter(myclass)
  this_word_counts_dict = dict(word_counts)#stores words count in each class to measure likelihood
  for word in this_word_counts_dict:
      if word not in words_probablites[k]  and word not in stopWords:#and len(word) > 1
        #occurence=countWord(word,words)
        log_likelihood=(log((this_word_counts_dict[word]+1)/(word_counts_dict[word]+1)))
        words_probablites[k][word]=log_likelihood

  del this_word_counts_dict



8544


test naivee bayes


In [ ]:
final_sum= [[] for _ in range(5)] #will contian the probability of each sentence to be in each class
j=0
print(len(words_probablites[3]))
for i in range (len(classes)):
  sum1=priors[i]
  for sentence in sentences:
      words=sentence.split(' ')
      m=0
      for word in words:#measures the probabily of each sentence for a specific class
        #if len(word) > 1:#skip one letter words
          if word in words_probablites[i] :#check if word appeared before in this class
            sum1+=words_probablites[i][word]
          m+=1
      final_sum[i].append(sum1)

  j+=1
final_sum=np.array(final_sum)
print(final_sum[:,0])
max_indices = np.argmax(final_sum, axis=0)
max(max_indices)


7986
[-30.83016152 -19.34766005 -29.34274745 -22.01492355 -27.02556589]


3



```
# This is formatted as code
```

sklearn naivee bayes



In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
model=make_pipeline(CountVectorizer(min_df=0),MultinomialNB(alpha=1))
model.fit(sentences,scores)
labels = model.predict(sentences)
labels[0]

3

confusion matrix

In [84]:

class ConfusionMatrix:
  def __init__(self,actual,predicted):
    self.confMat=np.zeros((5,5))
    self.true=0
    self.false=0
    self.t0=0
    for k in range(len(actual)):#predicted row and actual column
      i=predicted[k]#predicted label
      j=actual[k]#actual label
      self.confMat[i][j]+=1
      if i == j:
        self.true+=1
      else:
        self.false+=1
  def accuracy(self):
    return self.true/(self.true+self.false)


  def precision(self):
    precisions = []
    for i in range(len(self.confMat)):
        true_positives = self.confMat[i][i]
        false_positives = sum(self.confMat[j][i] for j in range(len(self.confMat)) if j != i)
        if true_positives + false_positives == 0:
            precisions.append(0)
        else:
            precisions.append(true_positives / (true_positives + false_positives))
    return precisions
  def recall(self):
    recall = []
    for i in range(len(self.confMat)):
        true_positives = self.confMat[i][i]
        false_negative = sum(self.confMat[i,j] for j in range(len(self.confMat)) if j != i)
        if true_positives + false_negative == 0:
            recall.append(0)
        else:
            recall.append(true_positives / (true_positives + false_negative))
    return recall
  def F1Score(self):
    precision = np.array(self.precision())
    recall = np.array(self.recall())
    f1_scores = []
    for i in range(len(precision)):
        if precision[i] + recall[i] == 0:
            f1_scores.append(0)  # Handle the case of both precision and recall being zero
        else:
            f1_scores.append(2 * precision[i] * recall[i] / (precision[i] + recall[i]))
    return f1_scores
  def percisionMacro(self):
    precision = np.array(self.precision())
    return sum(precision)/5
  def recallMacro(self):
    recall = np.array(self.recall())
    return sum(recall)/5
  def f1Macro(self):
    F1Score = (self.F1Score())
    return sum(F1Score)/5

sklearnMethod=ConfusionMatrix(actual=scores,predicted=labels)
myMethod=ConfusionMatrix(actual=scores,predicted=max_indices)

print(f"my method accuracy = {myMethod.f1Macro()} sklearn accuracy = {sklearnMethod.accuracy()}")
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
f1=f1_score(max_indices,scores,average='macro')
recall = recall_score(max_indices,scores,average=None)
print(recall)
print(f1)

[0, 0, 0, 0.42726853984339014, 0]
my method accuracy = 0.08545370796867803 sklearn accuracy = 0.7928370786516854
[0.         0.         0.         0.27176843 0.        ]
0.08545370796867803


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Logistic Regression

Feature Representation

In [ ]:
from ctypes import BigEndianStructure
def generate_word_bi_grams(sentences):
    # Create a set to store unique word bi-grams
    unique_bi_grams =set()

    # Iterate through each sentence to extract word bi-grams
    for sentence in sentences:
        words = sentence.split()
        # words= [word for word in words if word not in stopWords ]
        bi_grams = zip(words, words[1:])
        unique_bi_grams.update(bi_grams)

    # Convert the set of unique word bi-grams to a sorted list

    return sorted(list(unique_bi_grams))

In [ ]:
bi_grams = generate_word_bi_grams(sentences)

print(bi_grams)

[('!', "'"), ('#', '3'), ('#', '9'), ('$', '1.8'), ('$', '20'), ('$', '40'), ('$', '50-million'), ('$', '8'), ('$', '9'), ('$', '99'), ('&', 'about'), ('&', 'augmented'), ('&', 'christian'), ('&', 'co.'), ('&', 'heart-rate-raising'), ('&', 'j'), ('&', 'paulette'), ('&', 'stitch'), ('&', 'tonto'), ('&', 'w'), ('&', 'z-boys'), ("'", '('), ("'", ')'), ("'", ','), ("'", '-'), ("'", '--'), ("'", '.'), ("'", '..'), ("'", '...'), ("'", '20,000'), ("'", ':'), ("'", ';'), ("'", '`'), ("'", 'a'), ("'", 'about'), ("'", 'according'), ("'", 'achievement'), ("'", 'actors'), ("'", 'advises'), ("'", 'affections'), ("'", 'ambitious'), ("'", 'amusing'), ("'", 'anarchy'), ("'", 'and'), ("'", 'anderson'), ("'", 'anguish'), ("'", 'anti-'), ("'", 'appeared'), ("'", 'applies'), ("'", 'are'), ("'", 'as'), ("'", 'at'), ("'", 'award'), ("'", 'based'), ("'", 'because'), ("'", 'best'), ("'", 'better'), ("'", 'big-screen'), ("'", 'bohemian'), ("'", 'book'), ("'", 'borscht'), ("'", 'bow'), ("'", 'but'), ("'", 'butl

In [ ]:
len(bi_grams)

80936

In [ ]:
def generate_sparse_feature_representation(sentences, unique_bi_grams):
    feature_representation = []
    # Iterate through each sentence to create the sparse feature vector
    i=0
    for sentence in sentences:
        words = sentence.split()
        bi_grams = set(zip(words, words[1:]))

        # Create a binary vector for the current sentence
        sentence_vector=[]
        for bi_gram in bi_grams:
          if bi_gram in unique_bi_grams:
            index = unique_bi_grams.index(bi_gram)
            sentence_vector.append(index)
        feature_representation.append(sentence_vector)
        i+=1

    return feature_representation

In [ ]:
features = generate_sparse_feature_representation(sentences, bi_grams)

In [ ]:
# each sentence has a vector of features of length
len(features)
# len(test_sen)
# features[2]

8544

sigmoid Function


In [ ]:
def sigmoid(x):
  return (1/(1+np.exp(-x)))
# turn scores to softmax matrix
def turnScores(scores):
  new_scores=np.zeros((len(scores),5))
  i=0
  for score in scores:
    new_scores[i,score]=1
    i+=1
  return new_scores
def predict(weights,bias,x):
  z=np.dot(weights,x)

In [ ]:
# z = WX+B
b=np.zeros((5))#bias for each sentence
new_scores=turnScores(scores)
print(new_scores[0])
W=np.zeros((5,len(features[0])))
W.shape

[0. 0. 0. 1. 0.]


(5, 34)

In [ ]:
def multiClassLogistic(scores,features,learningRate,num_epochs,sample_size,num_of_samples):
  b=np.zeros((5))#bias for each sentence
  W=np.zeros((5,len(features[0])+1))
  for epoch in range(num_epochs):
      random_samples = np.random.uniform(0, len(features), 64)
      for sample in random_samples:
        X=features[:,sample]
        Y=scores[:,sample]



## IMplementation

In [ ]:
def turnScores(scores):
  new_scores=np.zeros((len(scores),5))
  i=0
  for score in scores:
    new_scores[i,score]=1
    i+=1
  return new_scores


y_one_hot = turnScores(scores)


In [ ]:
y_one_hot.shape

(8544, 5)

In [ ]:
num_features = len(bi_grams)
weights = np.random.rand(num_features, 5)
bias = np.zeros((1, 5))


In [ ]:
def softmax(logits):
    exp_logits = np.exp(logits - np.max(logits, axis=1, keepdims=True))
    return exp_logits / np.sum(exp_logits, axis=1, keepdims=True)


In [ ]:
def forward_pass(X, weights, bias):
    logits = np.dot(X, weights) + bias
    probabilities = softmax(logits)
    return probabilities


In [ ]:
def cross_entropy_loss(y_true, probabilities):
    return -np.sum(y_true * np.log(probabilities + 1e-10)) / len(y_true)


In [ ]:
def backward_pass(X, y_true, probabilities, weights, bias, learning_rate):
    error = probabilities - y_true#probablites is y hat
    grad_weights = np.dot(X.T, error)#dl/dw
    grad_bias = np.sum(error, axis=0, keepdims=True)#dl/db

    weights -= learning_rate * grad_weights
    bias -= learning_rate * grad_bias

    return weights, bias


In [ ]:
import random
num_epochs = 1000
learning_rate = 0.001

for epoch in range(num_epochs):
      X = []
      Y = []
      random_samples = random.sample(range(len(features)), 64)
      for sample in random_samples:
        sentence_vector = np.zeros(len(bi_grams))
        feature_list = features[sample]
        Y.append(y_one_hot[sample])
        for i in feature_list:
          sentence_vector[i] = 1
        X.append(sentence_vector)

      X_batch = np.array(X)
      y_batch = np.array(Y)

      # Forward pass
      probabilities = forward_pass(X_batch, weights, bias)

      # Compute loss
      loss = cross_entropy_loss(y_batch, probabilities)

      # Backward pass (Gradient Descent)
      weights, bias = backward_pass(X_batch, y_batch, probabilities, weights, bias, learning_rate)

      print(f'Epoch {epoch + 1}, Loss: {loss}')


In [ ]:

X = []
for sample in range(len(features)):
  sentence_vector = np.zeros(len(bi_grams))
  feature_list = features[sample]
  for i in feature_list:
    sentence_vector[i] = 1
  X.append(sentence_vector)

In [ ]:
X = np.array(X)
print(X.shape)

y = np.argmax(forward_pass(X, weights, bias),axis=1)

In [ ]:
y[0]
myMethod=ConfusionMatrix(actual=scores,predicted=max_indices)
print(myMethod.accuracy())